In [2]:
# 구글 드라이브 
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings(action='ignore')

Mounted at /content/drive


## AutoML

In [ ]:
#!pip uninstall markupsafe
!pip install pycaret==2.3.10 markupsafe==2.0.1 pyyaml==5.4.1

In [ ]:
import jinja2
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

import xgboost as xgb
import random

import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')


In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(SEED) # Seed 고정

In [3]:
data = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/train_test/ver_0804/train_0804.csv')

In [4]:
data.shape

(49674, 250)

In [7]:
data.columns

Index(['yyyymmdd', 'area', 'sex', 'frequency', 'dd_mes', 'sum_dpth_fhsc',
       'dd_mefs', 'ssrate', 'ss_dur', 'sum_ss_hr',
       ...
       'act2', 'act3', 'act4', 'act5', 'act6', 'act7', 'drink', 'risky_drink',
       'self_obesity', 'healthy'],
      dtype='object', length=250)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data.head()

,yyyymmdd,area,sex,frequency,dd_mes,sum_dpth_fhsc,dd_mefs,ssrate,ss_dur,sum_ss_hr,min_tg,avg_pa,max_pa,min_pa,avg_pv,max_pv,min_pv,avg_rhm,min_rhm,sum_rn,sum_rn_dur,mi10_max_rn,hr1_max_rn,hr6_max_rn,n9_9_rn,avg_ta,max_ta,min_ta,avg_ws,max_ws,max_ins_ws,dd_mes_lag1,sum_dpth_fhsc_lag1,dd_mefs_lag1,ssrate_lag1,ss_dur_lag1,sum_ss_hr_lag1,min_tg_lag1,avg_pa_lag1,max_pa_lag1,min_pa_lag1,avg_pv_lag1,max_pv_lag1,min_pv_lag1,avg_rhm_lag1,min_rhm_lag1,sum_rn_lag1,sum_rn_dur_lag1,mi10_max_rn_lag1,hr1_max_rn_lag1,hr6_max_rn_lag1,n9_9_rn_lag1,avg_ta_lag1,max_ta_lag1,min_ta_lag1,avg_ws_lag1,max_ws_lag1,max_ins_ws_lag1,dd_mes_lag2,sum_dpth_fhsc_lag2,dd_mefs_lag2,ssrate_lag2,ss_dur_lag2,sum_ss_hr_lag2,min_tg_lag2,avg_pa_lag2,max_pa_lag2,min_pa_lag2,avg_pv_lag2,max_pv_lag2,min_pv_lag2,avg_rhm_lag2,min_rhm_lag2,sum_rn_lag2,sum_rn_dur_lag2,mi10_max_rn_lag2,hr1_max_rn_lag2,hr6_max_rn_lag2,n9_9_rn_lag2,avg_ta_lag2,max_ta_lag2,min_ta_lag2,avg_ws_lag2,max_ws_lag2,max_ins_ws_lag2,dd_mes_lag3,sum_dpth_fhsc_lag3,dd_mefs_lag3,ssrate_lag3,ss_dur_lag3,sum_ss_hr_lag3,min_tg_lag3,avg_pa_lag3,max_pa_lag3,min_pa_lag3,avg_pv_lag3,max_pv_lag3,min_pv_lag3,avg_rhm_lag3,min_rhm_lag3,sum_rn_lag3,sum_rn_dur_lag3,mi10_max_rn_lag3,hr1_max_rn_lag3,hr6_max_rn_lag3,n9_9_rn_lag3,avg_ta_lag3,max_ta_lag3,min_ta_lag3,avg_ws_lag3,max_ws_lag3,max_ins_ws_lag3,SO2,CO,O3,NO2,PM10,S02_lag1,CO_lag1,O3_lag1,NO2_lag1,PM10_lag1,S02_lag2,CO_lag2,O3_lag2,NO2_lag2,PM10_lag2,S02_lag3,CO_lag3,O3_lag3,NO2_lag3,PM10_lag3,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,rainfall_24_shift,snow_24_shift,temp_24_shift,temp_48_shift,humid_24_shift,humid_48_shift,max_temp_24_shift,min_temp_24_shift,wind_24_shift,wind_48_shift,pop_over50,num_back,심정지,뇌졸중_진단,심장병_진단,고혈압_진단,이상지질혈증_진단,뇌졸중_약물치료,심장병_약물치료,고혈압_약물치료,이상지질혈증_약물치료,뇌졸중_가족력,심장병_가족력,고혈압_가족력,day1,smoking,day_mean_smoke_over1,A03_max,A03_min,A03_mean,A03_range,A03_iqr,A03_max_24,A03_min_24,A03_mean_24,A03_range_24,A03_iqr_24,A03_max_48,A03_min_48,A03_mean_48,A03_range_48,A03_iqr_48,A07_max,A07_min,A07_mean,A07_range,A07_iqr,D02_max,D02_min,D02_median,D02_max_24,D02_min_24,D02_median_24,D02_range_24,D02_max_48,D02_min_48,D02_median_48,D02_range_48,weekday_sin,weekday_cos,season_sin,season_cos,area_le,area_code,avg_pa_7일평균,max_pa_7일평균,min_pa_7일평균,avg_ta_7일평균,max_ta_7일평균,min_ta_7일평균,avg_rhm_7일평균,min_rhm_7일평균,max_ws_7일평균,max_ins_ws_7일평균,NO2_7일평균,avg_pa_14일평균,max_pa_14일평균,min_pa_14일평균,avg_ta_14일평균,max_ta_14일평균,min_ta_14일평균,avg_rhm_14일평균,min_rhm_14일평균,max_ws_14일평균,max_ins_ws_14일평균,NO2_14일평균,요일,주말,season,Mon,diff_ta,큰일교차_횟수,drink1,drink2,drink3,drink4,act0,act1,act2,act3,act4,act5,act6,act7,drink,risky_drink,self_obesity,healthy
0,2012-01-01,강원,1,3,10.136053,2.155943,1.303480,35.856285,9.632506,3.444997,-9.869523,1007.235630,1009.325584,1004.004358,3.126835,4.304677,2.415617,63.236009,40.360801,1.319425,3.795499,2.565648,4.369274,0.605217,0.768841,-2.420576,2.045879,-6.453899,1.780413,5.154082,8.943097,0.425440,0.053928,0.067554,53.706797,9.603948,5.157309,-10.711643,1010.952481,1014.182062,1008.379417,2.770426,3.492906,1.906770,60.089753,35.394057,5.922542,5.522200,2.405613,4.657815,4.245112,1.807018,-2.751374,1.921806,-7.806485,1.372664,3.322841,5.392499,8.157597,1.659180,0.039479,68.229108,9.592791,6.542103,-9.870923,1014.689511,1016.430323,1013.155854,3.430026,4.089441,2.732705,72.183833,46.157230,3.723311,1.857098,1.381549,2.694366,2.072126,0.761383,-3.316662,2.957736,-7.432780,1.202674,3.046707,4.717232,0.600455,0.438470,0.342631,45.458230,9.590896,4.363502,-8.519633,1011.930912,1014.123420,1009.947116,4.112360,5.218578,2.733617,73.678905,35.325765,8.843681,3.142044,1.818329,5.026921,8.630879,3.855059,-1.116461,3.030391,-4.974100,1.262846,3.634993,5.607580,0.011632,1.124709,0.016572,0.024203,93.238136,0.001753,0.246584,0.001978,0.006568,22.574379,0.001927,0.258629,0.001775,0.006279,19.798797,0.003004,0.282738,0.001703,0.007072,21.683691,14.00509,11.402662,-2.769171,-2.780795,62.145093,54.361839,2.777686,-

In [ ]:
model = setup(data, 
              target = 'frequency', 
              
              normalize = True,
              normalize_method = "minmax",

              ignore_features = ['area_le','area_code','요일'],
              categorical_features = ['num_back','day1','season','Mon','주말'],
              date_features = ['yyyymmdd'],
              
              
              session_id=SEED,
              use_gpu=True)

,Description,Value
0,session_id,42
1,Target,frequency
2,Original Data,"(49674, 250)"
3,Missing Values,False
4,Numeric Features,237
5,Categorical Features,8
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(34771, 263)"


In [ ]:
br_model = create_model("br",fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9068,1.5150,1.2308,0.4845,0.4803,0.4498
1,0.8975,1.4782,1.2158,0.4691,0.4810,0.4529
2,0.8897,1.4331,1.1971,0.4554,0.4817,0.4505
3,0.8801,1.4524,1.2051,0.4583,0.4856,0.4490
4,0.9111,1.4923,1.2216,0.4661,0.4924,0.4624
5,0.8914,1.4317,1.1965,0.4847,0.4837,0.4431
6,0.9058,1.5227,1.2340,0.4470,0.4870,0.4541
7,0.9154,1.5709,1.2534,0.4672,0.4853,0.4624
8,0.9137,1.5520,1.2458,0.4407,0.4923,0.4746


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/train_test/ver_0804/test_0804.csv')

In [ ]:
prediction = predict_model(br_model, data=test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


In [ ]:
prediction

,yyyymmdd,area,sex,dd_mes,sum_dpth_fhsc,dd_mefs,ssrate,ss_dur,sum_ss_hr,min_tg,...,act3,act4,act5,act6,act7,drink,risky_drink,self_obesity,healthy,Label
0,2016-01-01,강원,1,25.262803,8.088057,6.832984,41.929419,9.633102,4.029989,-8.917460,...,0.11,0.05,0.05,0.02,0.03,57.3,15.8,31.6,23.2,1.975449
1,2016-01-01,강원,2,25.262803,8.088057,6.832984,41.929419,9.633102,4.029989,-8.917460,...,0.06,0.03,0.03,0.01,0.02,57.3,15.8,31.6,23.2,1.916427
2,2016-01-01,경기,1,3.140683,0.247502,0.263841,19.758979,9.668681,1.907674,-6.721469,...,0.09,0.04,0.04,0.01,0.02,61.0,14.7,28.0,30.2,5.884081
3,2016-01-01,경기,2,3.140683,0.247502,0.263841,19.758979,9.668681,1.907674,-6.721469,...,0.08,0.03,0.03,0.01,0.01,61.0,14.7,28.0,30.2,5.611710
4,2016-01-01,경남,1,0.415067,0.216473,0.155119,86.066663,9.814577,8.444625,-6.450657,...,0.10,0.04,0.04,0.02,0.02,59.6,15.4,28.1,22.0,2.983378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,제주,2,0.073358,0.220075,0.100000,67.536586,10.000000,6.753659,1.447092,...,0.06,0.02,0.02,0.01,0.01,58.4,16.4,28.7,25.2,0.095772
12440,2016-12-31,충남,1,1.619904,0.013797,0.006899,23.264737,9.705368,2.256730,-5.365824,...,0.09,0.04,0.03,0.01,0.02,56.5,14.6,28.8,26.9,0.943848
12441,2016-12-31,충남,2,1.619904,0.013797,0.006899,23.264737,9.705368,2.256730,-5.365824,...,0.06,0.02,0.03,0.01,0.01,56.5,14.6,28.8,26.9,0.877247
12442,2016-12-31,충북,1,1.096842,0.000000,0.000000,20.767888,9.700000,2.016882,-7.126462,...,0.09,0.04,0.03,0.01,0.02,58.0,15.9,28.8,26.6,0.760430


In [ ]:
prediction = prediction[['yyyymmdd','area','sex','Label']]

In [ ]:
prediction

,yyyymmdd,area,sex,Label
0,2016-01-01,강원,1,1.975449
1,2016-01-01,강원,2,1.916427
2,2016-01-01,경기,1,5.884081
3,2016-01-01,경기,2,5.611710
4,2016-01-01,경남,1,2.983378
...,...,...,...,...
12439,2016-12-31,제주,2,0.095772
12440,2016-12-31,충남,1,0.943848
12441,2016-12-31,충남,2,0.877247
12442,2016-12-31,충북,1,0.760430


In [ ]:
prediction=prediction.sort_values(["yyyymmdd","sex"])
prediction.head(10)

,yyyymmdd,area,sex,Label
0,2016-01-01,강원,1,1.975449
2,2016-01-01,경기,1,5.884081
4,2016-01-01,경남,1,2.983378
6,2016-01-01,경북,1,2.901997
8,2016-01-01,광주,1,1.666149
10,2016-01-01,대구,1,2.115431
12,2016-01-01,대전,1,1.606691
14,2016-01-01,부산,1,2.833338
16,2016-01-01,서울,1,4.777202
18,2016-01-01,세종,1,0.867587


In [ ]:
prediction.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
prediction

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.975449
2,2016-01-01,경기,1,5.884081
4,2016-01-01,경남,1,2.983378
6,2016-01-01,경북,1,2.901997
8,2016-01-01,광주,1,1.666149
...,...,...,...,...
12435,2016-12-31,전남,2,1.493083
12437,2016-12-31,전북,2,1.033761
12439,2016-12-31,제주,2,0.095772
12441,2016-12-31,충남,2,0.877247


In [ ]:
prediction[prediction["frequency"]<0]

,yyyymmdd,area,sex,frequency
52,2016-01-02,세종,1,-0.365377
53,2016-01-02,세종,2,-0.135666
86,2016-01-03,세종,1,-0.375493
96,2016-01-03,제주,1,-0.055408
87,2016-01-03,세종,2,-0.145782
...,...,...,...,...
12235,2016-12-25,제주,2,-0.043918
12360,2016-12-29,세종,1,-0.015376
12394,2016-12-30,세종,1,-0.136241
12428,2016-12-31,세종,1,-0.263316


In [ ]:
for i in range(0,prediction.shape[0]) :
  if prediction.iloc[i,3] < 0 :
    prediction.iloc[i,3] = 0

In [ ]:
prediction[prediction["frequency"]<0]

,yyyymmdd,area,sex,frequency


In [ ]:
prediction.to_csv('/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/autoML예림확인용.csv', index=False)